In [ ]:

# LOOK AT SOME STATS

series2 = df.price_usd[1:].values
for col in df.columns.values[1:-2]:
    series1 = df[col][:-1].values

    # Find indices where both series have non-missing values
    non_missing_indices = np.logical_not(np.isnan(series1) | np.isnan(series2))

    # Compute the correlation using non-missing values only
    corr_matrix = np.corrcoef(series1[non_missing_indices], series2[non_missing_indices])

    # Extract the correlation coefficient (off-diagonal element)
    corr_coef = corr_matrix[0, 1]
    print(col)
    print(f"Correlation coefficient: {corr_coef}")


In [ ]:
# TODO UPDATE THIS OLD MESSY SCRIPT TO BUILD PANELS AT HOURLY, DAILY, AND WEEKLY FREQ

In [ ]:
# TODO add USDC and USDT as macro variables for the price deviation from peg
# TODO add the cleaned exchange rate, ref rate, and cmc prices to the panel for exploratory data analysis to
# -show how far off you'd be by using this price as opposed to the actually available price
# -can note it'd be even worse for assets that i'm not looking as they'd be adversely selected toward lower liquidity


In [ ]:

# TODO PULL IN RAW CM PANEL TO SEE IF ANY CRITICAL VARIABLES HELPFUL TO CROSSCHECK? 
#      E.G. SUPPLY, MCAP, ADDRESSES, TRANSFERS

In [ ]:
# TODO DO I NEED TO GO BACK TO GET MORE DATA BECAUSE I CUT ASSETS DOWN TO WHEN THEY FIRST APPEAR IN THE PANEL WHEN I SHOULD ACTUALLY GET ALL HISTORICAL DATA FOR THEM ACK TO JULY 1 2016?
# -SAN I STILL HAVE?
# -COINAPI I STILL AHVE BUT I DONT THINK I NEED TO DO?
# -COINGECKO WOULD BE TRIVIAL BUT PRLY NOT NEEDED? SAME FOR CMC? CMC COSTS OR MAYBE I STILL AHVE?

In [ ]:
# TODO CONSIDER GOING BACK TO LEAVE STUFF MISSING FOR WHEN I COMBINE COLUMNS TO JUST USE NON MISSING VALUES
# TODO RECONFIRM COLUMN RANGES 
# TODO FIGURE OUT HOW TO COMPRESS DOWN THE COLUMNS TO FEWER NUMBER FOR VARIOUS CATEGORIES

In [ ]:


# EXPLORE RETURNS OF EQUAL- AND MCAP- WEIGHTED PORTFOLIOS

# Cut the panel down to just the assets of interest
asset_universe_unique = list(np.unique([asset 
                                        for sublist in token_universe_per_month 
                                        for asset in sublist]))
df = df[df.slug_cmc.isin(asset_universe_unique)]

# Drop rows that do not have previous day information
df = df.sort_values(by=['slug_cmc', 'date'], ignore_index=True)
df.loc[1:, 'day_diff'] = (df.date[1:].values - df.date[:-1]).values.astype('timedelta64[D]').astype(int)
df['day_diff2'] = df.day_diff.shift(-1)
num_rows = df[df.day_diff == 1].shape[0]
df = df[(df.day_diff == 1) | (df.day_diff2 == 1)]
assert(num_rows <= df.shape[0])
df = df.drop(['day_diff2'], axis=1)

# Calculate day over day return
df['r_t'] = df.groupby('slug_cmc')['usd_per_token_cmc'].apply(pd.Series.pct_change)
df = df[df.day_diff == 1]
tokens_to_drop = np.unique(df[df.r_t.isnull()].slug_cmc.values)
df = df[~df.slug_cmc.isin(tokens_to_drop)]
df = df.drop('day_diff', axis=1)

# Cut down to time period of interest
df = df[df.date.dt.year >= 2016]
df = df[df.date.dt.year <= 2022]

# Ensure no missings
assert(0 == df.isnull().sum().sum())

# Clean up index and resort
df = df.sort_values(by=['date', 'slug_cmc'], ignore_index=True)

# Calculate equal and mcap weighted returns by quarter
equal_df = pd.DataFrame()
mcap_df  = pd.DataFrame()
for i in range(1,len(dates)):
    # Set up dates and asset universe
    date = dates[i]
    date_plus_1mo = datetime.datetime.strptime(date, '%Y-%m-%d') + relativedelta(months=1)
    asset_universe = asset_universe_dict[date]

    # Subset to relevant data
    temp_df = df[(df.date >= date) & (df.date < date_plus_1mo)]
    temp_df = temp_df[temp_df.slug_cmc.isin(asset_universe)]

    # Form equal weighted returns
    temp_eq_df = temp_df.groupby('date')[['r_t']].mean()
    equal_df = pd.concat((equal_df, temp_eq_df))

    # Form mcap weighted returns
    temp_df['mcap_sum'] = temp_df.groupby('date')['usd_mcap_cmc'].transform('sum')
    temp_df['mcap_weight'] = temp_df.usd_mcap_cmc / temp_df.mcap_sum
    temp_df['mcap_r_t'] = temp_df.r_t * temp_df.mcap_weight
    temp_mcap_df = temp_df.groupby('date')[['mcap_r_t']].sum()
    mcap_df = pd.concat((mcap_df, temp_mcap_df))

# Ensure no missing
assert(0==equal_df.isnull().sum().values)
assert(0==mcap_df.isnull().sum().values)

# Report returns
print('equal weighted return:')
print(equal_df.apply(geometricAverageSimpleReturns, axis=0).values[0])
print('sharpe:')
print(np.mean(equal_df.r_t.values)/np.std(equal_df.r_t.values))
print('mcap weighted return:')
print(mcap_df.apply(geometricAverageSimpleReturns, axis=0).values[0])
print('sharpe:')
print(np.mean(mcap_df.mcap_r_t.values)/np.std(mcap_df.mcap_r_t.values))

# Form the returns by year
equal_df['year'] = equal_df.index.year
mcap_df['year'] = mcap_df.index.year
print('equal weighted return:')
print(equal_df.groupby('year').apply(geometricAverageSimpleReturns))
print('mcap weighted return:')
print(mcap_df.groupby('year').apply(geometricAverageSimpleReturns))
equal_df = equal_df.drop('year', axis=1)
mcap_df  = mcap_df.drop('year', axis=1)

# MOVE THESE NOTES TO CLEANING

# manually look through it to confirm they are legit tokens
# or maybe give this task to jacob
# or maybe schedule a time to do this with jacob so we 2x the speed

# Lets look to see if the 0.01% mcap rule is good for the entire time period

# Jan 1 2015 - $5B - $500k
# Jan 1 2016 - $7B - $700k
# Jan 1 2017 - $18B - $1.8M
# Jan 1 2018 - $600B - $60M
# Apr 1 2018 - $300B - $30M
# Jul 1 2018 - $250B - $25M
# Jan 1 2019 - $125B - $12M
# Apr 1 2019 - $145B - $14M
# Jul 1 2019 - $330B - $33M
# Oct 1 2019 - $220B - $22M
# Jan 1 2020 - $200B - $20M
# Apr 1 2020 - $175B - $17M
# Jul 1 2020 - $260B - $26M
# Oct 1 2020 - $340B - $34M
# Jan 1 2021 - $770B - $77M
# Apr 1 2021 - $1.9T - $190M
# Jul 1 2021 - $1.4T - $140M
# Oct 1 2021 - $2T - $200M


# loop over the asset universe to form just oos obs
df = pd.DataFrame()
for k,v in asset_universe_dict.items():
    start_date = datetime.datetime.strptime(k, '%Y-%m-%d')
    end_date = start_date+relativedelta(months=1)
    df = pd.concat((df, panel_df[(panel_df.date >= start_date) 
                                 & (panel_df.date < end_date)
                                 & panel_df.asset.isin(v)]))


## Overview of code to build the clean panel

* 1. [Helper functions](#helpers)
* 1.3/4. [Extract Stablecoin Data](#sc)
* 2. [Apply inclusion criteria to form asset universe](#inclusion)
* 3. [Import data and merge together panel](#merge)
* 4. [Clean panel](#clean)
    * 4.1. [Drop rows and columns](#drop)
    * 4.2  [Clean main columns: price, mcap, and volume](#main)
    * 4.3  [Combine columns](#combine)
    * 4.4  [Interpolate](#interpolate)
    * 4.5  [Clean remaining columns](#columns)
    * 4.6  [Subset to token universe](#subset)
    * 4.7  [Final cleaning steps and save](#save)


In [1]:
# IMPORT PACKAGES
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pickle

## 1. Helper functions <a class="anchor" id="helpers"></a>


In [2]:
# Helper function to ensure list of datetimes are consecutive
def determineLongestGapInDatetimeList(dates):
    max_gap = 0
    for idx in range(1, len(dates)):
        # checking for day difference greater than gap
        gap = (dates[idx] - dates[idx - 1]).astype('timedelta64[D]').astype(int)
        if gap > max_gap:
            max_gap = gap

    return max_gap

In [3]:
# Helper function to form geometric average of a numpy array or pandas series
def geometricAverageSimpleReturns(series):
    return np.product(series+1)**(1/len(series))-1

In [4]:
# Helper function to ensure one-to-one mapping between two ID columns
def isOneToOne(temp_df, col1, col2):
    first = temp_df.groupby(col1)[col2].size().sort_values().values
    second = temp_df.groupby(col2)[col1].size().sort_values().values
    assert(len(first) == len(second))
    return len(first) == np.sum(first == second)

## 1 3/4. Pull out stablecoin data<a class="anchor" id="sc"></a>

In [5]:
# IMPORT CMC DATA
cw_df  = pd.read_pickle('../3-data/clean/cw.pkl')
cmc_df = pd.read_pickle('../3-data/derived/cmc_panel.pkl')

In [6]:
# SUBSET TO NEEDED COLUMNS
cw_df = cw_df[['cmc_id', 'slug_cmc']]
cmc_df = cmc_df[['date', 'cmc_id', 'usd_per_token_cmc', 'usd_mcap_cmc',
                 'usd_volume_24h_cmc']]
df = cmc_df.merge(cw_df,
                  on='cmc_id',
                  how='inner',
                  validate='many_to_one')

In [7]:
# SUBSET TO NEEDED ASSETS

# tether. cmc id 825. 'USDT' name.
# usdc. cmc name is 'USD Coin'. cmc id is 3408.
# binanceusd. cmc name is 'Binance USD'. cmc id is 4687.
# dai. cmc name is 'Dai'. cmc id is 4943.
# trueusd. cmc name is  'TrueUSD'. cmc id is 2563.
# paxdollar. cmc name is 'Pax Dollar'. cmc id is 3330.
# pax gold. cmc name is 'PAX Gold'. cmc id is 4705.
# terrausd. cmc name is 'TerraUSD'. cmc id is 7129.
# gemini dollar. cmc name is 'Gemini Dollar'. 3306 is cmc id.
# synthetix usd. cmc name is 'sUSD'. cmc id is 2927.

id_list = [825, 3408, 4687, 4943,
           2563, 3330, 4705, 7129,
           3306, 2927]
df = df[df.cmc_id.isin(id_list)]

In [8]:
# CLEAN UP COLUMNS
df = df.drop('cmc_id', axis=1)
df = df.rename(columns={'slug_cmc': 'asset',
                        'usd_per_token_cmc': 'usd_per_token',
                        'usd_mcap_cmc': 'usd_mcap',
                        'usd_volume_24h_cmc': 'usd_volume_24h'})
df['sector_stablecoin'] = 1

In [9]:
# SAVE THE DATA
df.to_pickle('../3-data/derived/raw_stablecoin_panel.pkl')

## 2. Apply inclusion criteria to form asset universe <a class="anchor" id="inclusion"></a>


In [10]:
# IMPORT CW AND PANELS
cw_df  = pd.read_pickle('../3-data/clean/cw.pkl')
cmc_df = pd.read_pickle('../3-data/derived/cmc_panel.pkl')
coinapi_df = pd.read_pickle('../3-data/derived/coinapi_panel.pkl')
cmc_macro_df = pd.read_pickle('../3-data/derived/cmc_macro.pkl')

In [11]:
# MERGE PANELS TOGETHER
print('Coinapi merge...')
nrows_b4 = coinapi_df.shape[0]
coinapi_df = coinapi_df.merge(cw_df[~cw_df.coinapi_id.isnull()][['cmc_id', 'coinapi_id']],
                              on='coinapi_id',
                              how='inner',
                              validate='many_to_one')
print('Lost '+str(nrows_b4-coinapi_df.shape[0])+' rows of '+str(nrows_b4)+' rows that didn\'t have cmc id.\n')

df = cmc_df.merge(coinapi_df,
                  on=['date', 'cmc_id'],
                  how='outer',
                  validate='one_to_one')
del cmc_df, coinapi_df

Coinapi merge...
Lost 42097 rows of 302275 rows that didn't have cmc id.



In [12]:
# MERGE ON CMC MACRO VARIABLES
nrows_b4 = df.shape[0]
df = df.merge(cmc_macro_df,
              on='date',
              how='left',
              validate='many_to_one')
assert(nrows_b4 == df.shape[0])
del cmc_macro_df

In [13]:
# MERGE ON CW VARIABLES
cw_df = cw_df.drop(['coinapi_id', 'gecko_id', 'cm_id',
                    'messari_id', 'san_id', 'symbol_messari',
                    'name_san', 'ticker_san',
                    'name_cmc', 'symbol_cmc'], axis=1)
assert(cw_df.slug_cmc.is_unique)

nrows_b4 = df.shape[0]
df = df.merge(cw_df,
              on='cmc_id',
              how='inner',
              validate='many_to_one')
assert(nrows_b4 == df.shape[0])
del cw_df

In [14]:
# SORT
df = df.sort_values(by=['date', 'slug_cmc'], ignore_index=True)

In [15]:
# MANUALLY DROP TOKENS
# NOTES: ampleforth is a stablecoin, pax gold is a gold stablecoin, index, and wrapped tokens
wrapped_tokens_to_drop = ['ampleforth', 'cryptoindex-com-100', 'pax-gold',
                          'wrapped-centrifuge', 'wrapped-luna-token', 'wrapped-ncg', 'wrapped-nxm']
df = df[~df.slug_cmc.isin(wrapped_tokens_to_drop)]

In [16]:
# CUT TOKEN UNIVERSE DOWN TO TOKENS WITH COINAPI ID
# -COINAPI PULL ONLY PULLED TOKENS WITH MARKET ON MAJOR US EXCHANGE
# -NOTE: I DO THIS IN THIS ODD WAY AS CMC HAS DATA EARLIER THAN COINAPI
print(df.shape)
coinapi_token_universe = np.unique(df[~df.coinapi_id.isnull()].cmc_id.values)
df = df[df.cmc_id.isin(coinapi_token_universe)]
print(df.shape)

(5137841, 145)
(471391, 145)


In [17]:
# DROP ROWS WHERE COINAPI AND CMC DATA ARE WILDLY DIFFERENT
df = df[~(np.abs(df.usd_per_token_cmc-df.usd_per_token_coinapi)/df.usd_per_token_cmc > 0.25)].reset_index(drop=True)

# FORM PRICE, MCAP, & VOLUME COLUMNS FROM CMC AND COINAPI DATA
df['usd_per_token'] = np.nan
df.loc[df.usd_per_token_cmc.isnull(), 'usd_per_token'] = df.usd_per_token_coinapi
df.loc[df.usd_per_token_coinapi.isnull(), 'usd_per_token'] = df.usd_per_token_cmc
df.loc[(~df.usd_per_token_cmc.isnull()) & (~df.usd_per_token_coinapi.isnull()), 
       'usd_per_token'] = (((df.usd_per_token_cmc*df.usd_volume_24h_cmc) + 
                            (df.usd_per_token_coinapi*df.usd_volume_24h_coinapi)) / 
                           (df.usd_volume_24h_cmc + df.usd_volume_24h_coinapi))
df['usd_volume_24h'] = np.nan
df.loc[df.usd_volume_24h_cmc.isnull(), 'usd_volume_24h'] = df.usd_volume_24h_coinapi
df.loc[df.usd_volume_24h_coinapi.isnull(), 'usd_volume_24h'] = df.usd_volume_24h_cmc
df.loc[(~df.usd_volume_24h_cmc.isnull()) & (~df.usd_volume_24h_coinapi.isnull()),
       'usd_volume_24h'] = df.usd_volume_24h_cmc
df['usd_mcap'] = np.nan
df.loc[df.usd_mcap_cmc.isnull(), 'usd_mcap'] = df.usd_mcap_coinapi
df.loc[df.usd_mcap_coinapi.isnull(), 'usd_mcap'] = df.usd_mcap_cmc
df.loc[df.usd_mcap_cmc == 0, 'usd_mcap'] = df.usd_mcap_coinapi
df.loc[(np.abs(df.usd_mcap_cmc-df.usd_mcap_coinapi)/df.usd_mcap_cmc > 100),
       'usd_mcap'] = df.usd_mcap_coinapi
df.loc[(np.abs(df.usd_mcap_cmc-df.usd_mcap_coinapi)/df.usd_mcap_cmc < 100),
       'usd_mcap'] = (df.usd_mcap_cmc+df.usd_mcap_coinapi)/2

# DROP ROWS WHERE RANK, PRICE AND MCAP ARE MISSING
df = df[~df.usd_per_token.isnull()]
df = df[~df.usd_mcap.isnull()]
df = df[~df.rank_cmc.isnull()]

In [18]:
# ENSURE NO MISSING IN CRITICAL COLS
assert(0==df.slug_cmc.isnull().sum())
assert(0==df.cmc_id.isnull().sum())
assert(0==df.date.isnull().sum())
assert(0==df.usd_per_token.isnull().sum())
assert(0==df.usd_volume_24h.isnull().sum())
assert(0==df.usd_mcap.isnull().sum())
assert(0==df.rank_cmc.isnull().sum())

# ENSURE THERE IS A ONE TO ONE MAPPING BETWEEN KEY COLUMNS
assert(isOneToOne(df[~df.coinapi_id.isnull()], 'coinapi_id', 'cmc_id'))
assert(isOneToOne(df[~df.coinapi_id.isnull()], 'coinapi_id', 'slug_cmc'))
assert(isOneToOne(df, 'slug_cmc', 'cmc_id'))

# ENSURE THERE ARE NO CMC ID'S WITH MISSING COINAPI ID'S THAT WERE NOT IN THE COINAPI UNIVERSE
# NOTE: iow, we have coinapi tokens that were not in the coinapi set
cmc_ids_missing_coinapi_id = np.unique(df[df.coinapi_id.isnull()].cmc_id.values)
assert(0==len(set(cmc_ids_missing_coinapi_id).difference(set(coinapi_token_universe))))

In [19]:
# APPLY VOLUME AND MCAP FILTERS
dates = ['2016-01-01', '2016-04-01', '2016-07-01', '2016-10-01',       
         '2017-01-01', '2017-04-01', '2017-07-01', '2017-10-01',
         '2018-01-01', '2018-04-01', '2018-07-01', '2018-10-01',
         '2019-01-01', '2019-04-01', '2019-07-01', '2019-10-01',
         '2020-01-01', '2020-04-01', '2020-07-01', '2020-10-01',
         '2021-01-01', '2021-04-01', '2021-07-01', '2021-10-01',]
token_universe_per_quarter = []

for i in range(len(dates)):
    date = dates[i]
    print(date)
    print('\n')

    # Figure out date three months prior
    date_3_m_prior = datetime.strptime(date, '%Y-%m-%d') - relativedelta(months=3)

    # Subset to dataframe of days leading up to first day of quarter
    temp_df = df[(df.date <= date) & (df.date >= date_3_m_prior)].copy()

    # Obtain list of tokens to consider
    assets_included = list(np.unique(temp_df[temp_df.date == date].slug_cmc.values))

    # Figure out tokens removed due to insuff data
    # Note: 34 days ensures at least 4 weeks of data starting on a Sunday
    asset_ns_df = temp_df.groupby('slug_cmc').size()
    assets_lost_given_insuff_data = list(asset_ns_df[asset_ns_df < 34].index.values)
    print('Coins lost given less than 34 days of data:')
    print(np.unique(assets_lost_given_insuff_data))
    print('\n')
    for asset in assets_lost_given_insuff_data:
        if asset in assets_included:
            assets_included.remove(asset)   

    # Figure out tokens removed due to volume threshold
    temp_vol_df = temp_df.groupby('slug_cmc').usd_volume_24h.mean()
    assets_lost_given_insuff_vol = list(temp_vol_df[temp_vol_df < 10000].index.values)
    print('Coins lost given less than \$50,000 per day of average volume:')
    print(np.unique(assets_lost_given_insuff_vol))
    print('\n')
    for asset in assets_lost_given_insuff_vol:
        if asset in assets_included:
            assets_included.remove(asset)

    # Figure out tokens removed due to mcap threshold
    mcap_threshold = 0.00001*temp_df[temp_df.date == date].total_market_cap_cmc.values[0]
    print('MCAP threshold is: ' + str(round(mcap_threshold, 0)))
    temp_mcap_df = temp_df.groupby('slug_cmc').usd_mcap.mean()
    assets_lost_given_mcap_threshold = list(temp_mcap_df[temp_mcap_df < mcap_threshold].index.values)
    print('Coins lost given less than mcap threshold:')
    print(np.unique(assets_lost_given_mcap_threshold))
    print('\n')
    for asset in assets_lost_given_mcap_threshold:
        if asset in assets_included:
            assets_included.remove(asset)
            
    # Figure out tokens removed due to cmc rank threshold
    assets_below_rank_threshold = np.unique(temp_df[temp_df.rank_cmc > 1000].slug_cmc.values)
    print('Coins removed due to being below CMC rank of 1000:')
    for asset in assets_below_rank_threshold:
        if asset in assets_included:
            print(asset)
            assets_included.remove(asset)
    print('\n')

    # Report out new tokens ever
    print('New tokens that we have never had are ')
    if (date != '2016-01-01'):
        all_tokens = []
        for j in range(i-1,-1,-1):
            all_tokens += token_universe_per_quarter[j]
        print(np.unique(set(assets_included).difference(set(all_tokens))))
    else:
        print(np.unique(assets_included))
    print('\n')

    # Report out tokens lost from last quarter
    if date != '2016-01-01':
        print('Tokens lost from last quarter are ')
        print(set(token_universe_per_quarter[i-1]).difference(set(assets_included)))
        print('\n')

    # Report out tokens for this quarter
    print('This quarter\'s tokens are:')
    print(np.unique(assets_included))
    print(len(assets_included))
    print('\n\n\n')

    # Add tokens to list
    token_universe_per_quarter.append(list(np.unique(assets_included)))

2016-01-01


Coins lost given less than 34 days of data:
[]


Coins lost given less than \$50,000 per day of average volume:
['nem' 'siacoin']


MCAP threshold is: 70905.0
Coins lost given less than mcap threshold:
[]


Coins removed due to being below CMC rank of 1000:


New tokens that we have never had are 
['bitcoin' 'dash' 'digibyte' 'dogecoin' 'ethereum' 'litecoin' 'monero'
 'stellar' 'xrp']


This quarter's tokens are:
['bitcoin' 'dash' 'digibyte' 'dogecoin' 'ethereum' 'litecoin' 'monero'
 'stellar' 'xrp']
9




2016-04-01


Coins lost given less than 34 days of data:
[]


Coins lost given less than \$50,000 per day of average volume:
['pivx']


MCAP threshold is: 81114.0
Coins lost given less than mcap threshold:
['pivx']


Coins removed due to being below CMC rank of 1000:


New tokens that we have never had are 
[{'siacoin', 'decred', 'nem'}]


Tokens lost from last quarter are 
set()


This quarter's tokens are:
['bitcoin' 'dash' 'decred' 'digibyte' 'dogecoin' 'ethereum' 'li

Coins lost given less than 34 days of data:
['bionic' 'bitcoin-diamond' 'cardstack' 'eden' 'energi' 'fortknoxster'
 'morpheus-network' 'playgroundz' 'quant' 'ulord']


Coins lost given less than \$50,000 per day of average volume:
['adshares' 'energi' 'playgroundz' 'pluton']


MCAP threshold is: 2232786.0
Coins lost given less than mcap threshold:
['aigang' 'bionic' 'dero' 'energi' 'ergo' 'fortknoxster' 'iungo' 'kryll'
 'lala-world']


Coins removed due to being below CMC rank of 1000:
blocktrade-token
bounty0x
locicoin
nix
travala


New tokens that we have never had are 
[{'cosmo-coin', 'mainframe', 'constellation', 'gochain', 'pundi-x', 'xyo', 'iotex', 'oneledger', 'quarkchain', 'nimiq', 'effect-ai', 'nkn', 'merculet', 'tezos', 'vechain', 'omix'}]


Tokens lost from last quarter are 
{'lala-world', 'dero', 'iungo', 'aigang', 'bounty0x', 'adshares'}


This quarter's tokens are:
['0x' 'aave-old' 'adbank' 'adx-net' 'aelf' 'aion' 'alphacat' 'amber'
 'aragon' 'arcblock' 'asch' 'augur' 'ax

Coins lost given less than 34 days of data:
['decision-token' 'kusama' 'sharetoken']


Coins lost given less than \$50,000 per day of average volume:
['adshares' 'aigang' 'axpr-token' 'bionic' 'bounty0x' 'cargox' 'cashaa'
 'clearpoll' 'decision-token' 'deeponion' 'effect-ai' 'electrifyasia'
 'flixxo' 'iungo' 'kryll' 'lala-world' 'locicoin' 'mobius' 'naga' 'omix'
 'opacity' 'playgroundz' 'xyo' 'zeepin']


MCAP threshold is: 1809651.0
Coins lost given less than mcap threshold:
['adbank' 'adshares' 'aigang' 'alphacat' 'amino-network' 'axpr-token'
 'bionic' 'bounty0x' 'cappasity' 'cardstack' 'cargox' 'carvertical'
 'clearpoll' 'datum' 'datx' 'decision-token' 'effect-ai' 'electrifyasia'
 'ether-1' 'experty' 'ferrum-network' 'flixxo' 'fortknoxster' 'hyprr'
 'iht-real-estate-protocol' 'iungo' 'jarvis' 'kryll' 'lala-world' 'lcx'
 'locicoin' 'medical-chain' 'merculet' 'morpheus-labs' 'naga' 'omix'
 'opacity' 'playgroundz' 'pluton' 'sharetoken' 'sophiatx' 'sparkpoint'
 'zeepin']


Coins removed 

Coins lost given less than 34 days of data:
['alpaca-finance' 'amino-network' 'anchor-protocol' 'antimatter' 'babb'
 'chain-guardians' 'change' 'dora-factory' 'edge' 'glitch'
 'gspi-governance' 'hapi-one' 'ideaology' 'illuvium' 'konomi-network'
 'labs-group' 'lattice-token' 'locicoin' 'myneighboralice' 'oddz' 'omix'
 'radicle' 'radix' 'smartkey' 'strike' 'symbol' 'taraxa' 'tower-token'
 'xeonbit-token']


Coins lost given less than \$50,000 per day of average volume:
['adshares' 'aigang' 'bionic' 'change' 'cosmo-coin' 'edge' 'flixxo'
 'locicoin' 'omix' 'playgroundz' 'poet' 'sophiatx' 'trueflip' 'ulord'
 'xeonbit-token']


MCAP threshold is: 18797921.0
Coins lost given less than mcap threshold:
['adbank' 'adshares' 'aigang' 'alchemy-pay' 'alpaca-finance' 'alphacat'
 'amino-network' 'animalgo' 'arcblock' 'asch' 'axpr-token' 'babb' 'bionic'
 'blockmason' 'bolt' 'bounty0x' 'bread' 'bumo' 'cappasity' 'cardstack'
 'cargox' 'carvertical' 'caspian' 'change' 'clintex-cti' 'coinpoker'
 'cosmo-co

In [20]:
# BUILD TOKEN UNIVERSE DICTIONARY
asset_universe_dict = {}
for i in range(len(dates)):
    asset_universe_dict[dates[i]] = token_universe_per_quarter[i]

In [21]:
# EXPLORE RETURNS OF EQUAL- AND MCAP- WEIGHTED PORTFOLIOS

# Cut the panel down to just the assets of interest
asset_universe_unique = list(np.unique([asset 
                                        for sublist in token_universe_per_quarter 
                                        for asset in sublist]))
df = df[df.slug_cmc.isin(asset_universe_unique)]

# Keep columns of interest
df = df[['date', 'slug_cmc', 'usd_per_token', 'usd_mcap', 'usd_volume_24h']]

# Drop rows that do not have previous day information
df = df.sort_values(by=['slug_cmc', 'date'], ignore_index=True)
df.loc[1:, 'day_diff'] = (df.date[1:].values - df.date[:-1]).values.astype('timedelta64[D]').astype(int)
df['day_diff2'] = df.day_diff.shift(-1)
num_rows = df[df.day_diff == 1].shape[0]
df = df[(df.day_diff == 1) | (df.day_diff2 == 1)]
assert(num_rows <= df.shape[0])
df = df.drop(['day_diff2'], axis=1)

# Calculate day over day return
df['r_t'] = df.groupby('slug_cmc')['usd_per_token'].apply(pd.Series.pct_change)
df = df[df.day_diff == 1]
tokens_to_drop = np.unique(df[df.r_t.isnull()].slug_cmc.values)
df = df[~df.slug_cmc.isin(tokens_to_drop)]
df = df.drop('day_diff', axis=1)

# Cut down to time period of interest
df = df[df.date.dt.year >= 2016]
df = df[df.date.dt.year <= 2021]

# Ensure no missings
assert(0 == df.isnull().sum().sum())

# Clean up index and resort
df = df.sort_values(by=['date', 'slug_cmc'], ignore_index=True)

# Calculate equal and mcap weighted returns by quarter
equal_df = pd.DataFrame()
mcap_df  = pd.DataFrame()
for i in range(len(dates)):
    # Set up dates and asset universe
    date = dates[i]
    date_plus_3mo = datetime.strptime(date, '%Y-%m-%d') + relativedelta(months=3)
    asset_universe = asset_universe_dict[date]

    # Subset to relevant data
    temp_df = df[(df.date >= date) & (df.date < date_plus_3mo)]
    temp_df = temp_df[temp_df.slug_cmc.isin(asset_universe)]

    # Form equal weighted returns
    temp_eq_df = temp_df.groupby('date')[['r_t']].mean()
    equal_df = pd.concat((equal_df, temp_eq_df))

    # Form mcap weighted returns
    temp_df['mcap_sum'] = temp_df.groupby('date')['usd_mcap'].transform('sum')
    temp_df['mcap_weight'] = temp_df.usd_mcap / temp_df.mcap_sum
    temp_df['mcap_r_t'] = temp_df.r_t * temp_df.mcap_weight
    temp_mcap_df = temp_df.groupby('date')[['mcap_r_t']].sum()
    mcap_df = pd.concat((mcap_df, temp_mcap_df))

# Ensure no missing
assert(0==equal_df.isnull().sum().values)
assert(0==mcap_df.isnull().sum().values)

# Report returns
print('equal weighted return:')
print(equal_df.apply(geometricAverageSimpleReturns, axis=0).values[0])
print('sharpe:')
print(np.mean(equal_df.r_t.values)/np.std(equal_df.r_t.values))
print('mcap weighted return:')
print(mcap_df.apply(geometricAverageSimpleReturns, axis=0).values[0])
print('sharpe:')
print(np.mean(mcap_df.mcap_r_t.values)/np.std(mcap_df.mcap_r_t.values))

# Form the returns by year
equal_df['year'] = equal_df.index.year
mcap_df['year'] = mcap_df.index.year
print('equal weighted return:')
print(equal_df.groupby('year').apply(geometricAverageSimpleReturns))
print('mcap weighted return:')
print(mcap_df.groupby('year').apply(geometricAverageSimpleReturns))
equal_df = equal_df.drop('year', axis=1)
mcap_df  = mcap_df.drop('year', axis=1)

equal weighted return:
0.004176552238186915
sharpe:
0.11362512249549817
mcap weighted return:
0.003410722459926019
sharpe:
0.10372985806628685
equal weighted return:
           r_t      year
year                    
2016  0.004193       NaN
2017  0.015706 -1.000000
2018 -0.004921  0.124385
2019 -0.000152 -1.000000
2020  0.003962  0.121263
2021  0.006341 -1.000000
mcap weighted return:
      mcap_r_t      year
year                    
2016  0.002562       NaN
2017  0.012404 -1.000000
2018 -0.003802  0.124385
2019  0.001198 -1.000000
2020  0.004098  0.121263
2021  0.004035 -1.000000


In [22]:
# Save object and clear memory
with open('../3-data/clean/asset_universe_dates_and_lists.pkl', 'wb') as f:
    pickle.dump(asset_universe_dict, f)
del df

## 3. Import data and merge panel together <a class="anchor" id="merge"></a>


In [23]:
# IMPORT CW AND PANEL DATA
cw_df  = pd.read_pickle('../3-data/clean/cw.pkl')
cmc_df = pd.read_pickle('../3-data/derived/cmc_panel.pkl')
coinapi_df = pd.read_pickle('../3-data/derived/coinapi_panel.pkl')
coinapi_nonus_df = pd.read_pickle('../3-data/derived/coinapi_panel_nonus.pkl')
coingecko_df = pd.read_pickle('../3-data/derived/coingecko_panel.pkl')
cm_df = pd.read_pickle('../3-data/derived/coinmetrics_panel.pkl')
san_df = pd.read_pickle('../3-data/derived/santiment_panel.pkl')

In [24]:
# CONFIRM CMC ID AND SLUG ARE ONE TO ONE AND UNIQUE
assert(cw_df.slug_cmc.is_unique)
assert(isOneToOne(cw_df, 'slug_cmc', 'cmc_id'))

In [25]:
# CONFIRM THERE IS NO MISSING DATA IN PANELS
assert(0 == cmc_df.usd_per_token_cmc.isnull().sum())
assert(0 == coinapi_df.usd_per_token_coinapi.isnull().sum())

In [26]:
# MERGE CMC SLUG ONTO ALL PANELS
# AND SUBSET TO JUST THOSE IN THE TOKEN UNIVERSE
asset_universe_unique = list(np.unique([asset for sublist in asset_universe_dict.values()
                                              for asset in sublist]))

print('CMC merge...')
nrows_b4 = cmc_df.shape[0]
cmc_df = cmc_df.merge(cw_df[~cw_df.cmc_id.isnull()][['slug_cmc', 'cmc_id']],
                              on='cmc_id',
                              how='inner',
                              validate='many_to_one')
cmc_df = cmc_df.drop('cmc_id', axis=1)
tether_df = cmc_df[cmc_df.slug_cmc == 'tether'] # EXTRACT TETHER OBS FOR MACRO VAR
cmc_df = cmc_df[cmc_df.slug_cmc.isin(asset_universe_unique)]
print('Lost '+str(nrows_b4-cmc_df.shape[0])+' rows of '
      +str(nrows_b4)+' rows that didn\'t have cmc slug nor were in the token universe.\n')

print('Coinapi merge...')
nrows_b4 = coinapi_df.shape[0]
coinapi_df = coinapi_df.merge(cw_df[~cw_df.coinapi_id.isnull()][['slug_cmc', 'coinapi_id']],
                              on='coinapi_id',
                              how='inner',
                              validate='many_to_one')
coinapi_df = coinapi_df.drop('coinapi_id', axis=1)
coinapi_df = coinapi_df[coinapi_df.slug_cmc.isin(asset_universe_unique)]
print('Lost '+str(nrows_b4-coinapi_df.shape[0])+' rows of '
      +str(nrows_b4)+' rows that didn\'t have cmc slug nor were in the token universe.\n')

print('Coinapi non-US merge...')
nrows_b4 = coinapi_nonus_df.shape[0]
coinapi_nonus_df = coinapi_nonus_df.merge(cw_df[~cw_df.coinapi_id.isnull()][['slug_cmc', 'coinapi_id']],
                                          on='coinapi_id',
                                          how='inner',
                                          validate='many_to_one')
coinapi_nonus_df = coinapi_nonus_df.drop('coinapi_id', axis=1)
coinapi_nonus_df = coinapi_nonus_df[coinapi_nonus_df.slug_cmc.isin(asset_universe_unique)]
print('Lost '+str(nrows_b4-coinapi_nonus_df.shape[0])+' rows of '
      +str(nrows_b4)+' rows that didn\'t have cmc slug nor were in the token universe.\n')

print('Coingecko merge...')
nrows_b4 = coingecko_df.shape[0]
coingecko_df = coingecko_df.merge(cw_df[~cw_df.gecko_id.isnull()][['slug_cmc', 'gecko_id']],
                                  on='gecko_id',
                                  how='inner',
                                  validate='many_to_one')
coingecko_df = coingecko_df.drop('gecko_id', axis=1)
coingecko_df = coingecko_df[coingecko_df.slug_cmc.isin(asset_universe_unique)]
print('Lost '+str(nrows_b4-coingecko_df.shape[0])+' rows of '
      +str(nrows_b4)+' rows that didn\'t have cmc slug nor were in the token universe.\n')

print('Coinmetrics merge...')
nrows_b4 = cm_df.shape[0]
cm_df = cm_df.merge(cw_df[~cw_df.cm_id.isnull()][['slug_cmc', 'cm_id']],
                          on='cm_id',
                          how='inner',
                          validate='many_to_one')
cm_df = cm_df.drop('cm_id', axis=1)
cm_df = cm_df[cm_df.slug_cmc.isin(asset_universe_unique)]
print('Lost '+str(nrows_b4-cm_df.shape[0])+' rows of '
      +str(nrows_b4)+' rows that didn\'t have cmc slug nor were in the token universe.\n')

print('Santiment merge...')
nrows_b4 = san_df.shape[0]
san_df = san_df.merge(cw_df[~cw_df.san_id.isnull()][['slug_cmc', 'san_id']],
                            on='san_id',
                            how='inner',
                            validate='many_to_one')
san_df = san_df.drop('san_id', axis=1)
san_df = san_df[san_df.slug_cmc.isin(asset_universe_unique)]
print('Lost '+str(nrows_b4-san_df.shape[0])+' rows of '
      +str(nrows_b4)+' rows that didn\'t have cmc slug nor were in the token universe.\n')

CMC merge...
Lost 4711756 rows of 5136649 rows that didn't have cmc slug nor were in the token universe.

Coinapi merge...
Lost 71857 rows of 302275 rows that didn't have cmc slug nor were in the token universe.

Coinapi non-US merge...
Lost 158078 rows of 158078 rows that didn't have cmc slug nor were in the token universe.

Coingecko merge...
Lost 660084 rows of 944376 rows that didn't have cmc slug nor were in the token universe.

Coinmetrics merge...
Lost 82582 rows of 203224 rows that didn't have cmc slug nor were in the token universe.

Santiment merge...
Lost 1372421 rows of 2162457 rows that didn't have cmc slug nor were in the token universe.



In [27]:
# MERGE PANELS TOGETHER
df = cmc_df.merge(coinapi_df,
                  on=['date', 'slug_cmc'],
                  how='outer',
                  validate='one_to_one')
del cmc_df, coinapi_df
df = df.merge(coinapi_nonus_df,
             on=['date', 'slug_cmc'],
             how='outer',
             validate='one_to_one')
del coinapi_nonus_df
df = df.merge(coingecko_df,
             on=['date', 'slug_cmc'],
             how='outer',
             validate='one_to_one')
del coingecko_df
df = df.merge(cm_df,
             on=['date', 'slug_cmc'],
             how='outer',
             validate='one_to_one')
del cm_df
df = df.merge(san_df,
             on=['date', 'slug_cmc'],
             how='outer',
             validate='one_to_one')
del san_df

In [28]:
# MERGE ON CW VARIABLES
cw_df = cw_df.drop(['cmc_id', 'coinapi_id', 'gecko_id', 'cm_id',
                    'messari_id', 'san_id', 'symbol_messari',
                    'name_san', 'ticker_san',
                    'name_cmc', 'symbol_cmc'], axis=1)
assert(cw_df.slug_cmc.is_unique)

nrows_b4 = df.shape[0]
df = df.merge(cw_df,
              on='slug_cmc',
              how='inner',
              validate='many_to_one')
assert(nrows_b4 == df.shape[0])
del cw_df

In [29]:
# IMPORT MACRO DATA
cmc_macro_df = pd.read_pickle('../3-data/derived/cmc_macro.pkl')
cm_macro_df = pd.read_pickle('../3-data/derived/coinmetrics_macro.pkl')
san_macro_df = pd.read_pickle('../3-data/derived/santiment_macro.pkl')
mc_macro_df = pd.read_pickle('../3-data/derived/mccracken_macro.pkl')

In [30]:
# CLEAN TETHER DATA
tether_df = tether_df[['date', 'usd_per_token_cmc', 'usd_mcap_cmc', 'usd_volume_24h_cmc']]
tether_df = tether_df.rename(columns={'usd_per_token_cmc': 'usd_per_usdt_cmc',
                                      'usd_mcap_cmc': 'usd_mcap_usdt_cmc',
                                      'usd_volume_24h_cmc': 'usd_volume_24h_usdt_cmc'})
tether_df = tether_df[tether_df.date >= '2015-11-01']
full_dates_df = pd.DataFrame(data = {'date': 
                                     pd.date_range(pd.to_datetime(tether_df.date.values[0]), 
                                                   pd.to_datetime(tether_df.date.values[-1]), freq='D').values})
tether_df = tether_df.merge(full_dates_df,
                            on='date',
                            how='outer',
                            validate='one_to_one')
tether_df = tether_df.sort_values(by='date')
tether_df = tether_df.interpolate(method='pad')

# ENSURE PANEL AND MACRO PANELS HAVE EVERY DAY
assert(1 == determineLongestGapInDatetimeList(np.unique(df.date.values)))
assert(1 == determineLongestGapInDatetimeList(np.unique(cmc_macro_df.date.values)))
assert(1 == determineLongestGapInDatetimeList(np.unique(cm_macro_df.date.values)))
assert(1 == determineLongestGapInDatetimeList(np.unique(san_macro_df.date.values)))
assert(1 == determineLongestGapInDatetimeList(np.unique(mc_macro_df.date.values)))
assert(1 == determineLongestGapInDatetimeList(np.unique(tether_df.date.values)))

# ENSURE DATE IS UNIQUE
assert(cmc_macro_df.date.is_unique)
assert(cm_macro_df.date.is_unique)
assert(san_macro_df.date.is_unique)
assert(mc_macro_df.date.is_unique)
assert(tether_df.date.is_unique)

# MERGE ON MACRO DATA
nrows_b4 = df.shape[0]
df = df.merge(cmc_macro_df,
              on='date',
              how='left',
              validate='many_to_one')
assert(nrows_b4 >= df.shape[0])
del cmc_macro_df

nrows_b4 = df.shape[0]
df = df.merge(cm_macro_df,
              on='date',
              how='left',
              validate='many_to_one')
assert(nrows_b4 == df.shape[0])
del cm_macro_df

nrows_b4 = df.shape[0]
df = df.merge(san_macro_df,
              on='date',
              how='left',
              validate='many_to_one')
assert(nrows_b4 == df.shape[0])
del san_macro_df

nrows_b4 = df.shape[0]
df = df.merge(mc_macro_df,
              on='date',
              how='left',
              validate='many_to_one')
assert(nrows_b4 == df.shape[0])
del mc_macro_df

nrows_b4 = df.shape[0]
df = df.merge(tether_df,
              on='date',
              how='left', 
              validate='many_to_one')
assert(nrows_b4 == df.shape[0])
del tether_df

In [31]:
# SORT
df = df.sort_values(by=['date', 'slug_cmc'], ignore_index=True)

## 4. Clean panel <a class="anchor" id="clean"></a>


### 4.1. Drop rows and columns <a class="anchor" id="drop"></a>


In [32]:
# KEEP ONLY ROWS WHERE WE HAVE A COINAPI PRICE OR A CMC PRICE
# NOTE: coinapi should be when it starts as tradable but i am expanding a bit with CMC
df = df[~df.usd_per_token_cmc.isnull() | ~df.usd_per_token_coinapi.isnull()]
df = df.reset_index(drop=True)

In [33]:
# DROP COLUMNS WITH TOO MANY MISSING OBS

# Drop coinapi mcap as it is really from CMC so it is duplicated
df = df.drop('usd_mcap_coinapi', axis=1)

# Drop coinapi_nonus as it did not add any data
df = df.drop(['usd_per_token_coinapi_nonus', 'usd_volume_24h_coinapi_nonus',
              'daily_avg_marketcap_usd_san'], axis=1)

# Drop this column as san has a end of day price that is better
df = df.drop(['daily_avg_price_usd_san', 'daily_trading_volume_usd_san'], axis=1)

# Drop cols if above 50% missing
# note: I manually looked at cols to drop/keep
nrows = df.shape[0]
cols_to_drop = []
for col in df.columns.values:
    missing = df[col].isnull().sum()
    if missing/nrows > 0.5068:
        cols_to_drop.append(col)
        
cols_to_keep_above_missing_threshold = ['consensus_pow_messari', 'prct_premine_messari', 
                                        'prct_founder_premine_messari',
                                        'transactions_count_san', 'mean_dollar_invested_age_san',
                                        'mvrv_usd_san', 'age_destroyed_change_7d_san',
                                        'supply_outside_exchanges_san', 'supply_on_exchanges_san',
                                        'nvt_transaction_volume_san', 'nvt_san',
                                        'velocity_san', 'dormant_circulation_90d_san',
                                        'payments_count_san', 'transaction_volume_usd_san',
                                        'age_consumed_san',  'transaction_volume_san',
                                        'daily_active_addresses_san', 'mean_age_san',
                                        'active_addresses_24h_san', 'mean_realized_price_usd_san',
                                        'total_supply_in_profit_san', 'realized_value_usd_san', 
                                        'circulation_san']

for col in cols_to_keep_above_missing_threshold:
    cols_to_drop.remove(col)

df = df.drop(cols_to_drop, axis=1)

### 4.2. Clean main columns: price, mcap, and volume <a class="anchor" id="main"></a>


In [34]:
# CLEAN PRICE COLUMNS

# Set any zero prices to missing
df.loc[df.usd_per_token_coinapi == 0, 'usd_per_token_coinapi'] = np.nan
df.loc[df.usd_per_token_coinapi == 0, 'usd_per_token_cmc'] = np.nan
df.loc[df.usd_per_token_coinapi == 0, 'usd_per_token_coingecko'] = np.nan
df.loc[df.usd_per_token_coinapi == 0, 'usd_per_token_san'] = np.nan

# Create single price column
df['usd_per_token'] = df.usd_per_token_cmc

# Fix missing values of usd_per_token_cmc
df.loc[df.usd_per_token_cmc.isnull(), 'usd_per_token'] = df.usd_per_token_coinapi
assert(0==df.usd_per_token.isnull().sum())
df.loc[df.usd_per_token_cmc.isnull() &
       ~df.usd_per_token_coinapi.isnull() &
       ~df.usd_per_token_san.isnull() &
       ((df.usd_per_token_coinapi/df.usd_per_token_san > 1.2) | 
        (df.usd_per_token_coinapi/df.usd_per_token_san < 0.8)), 'usd_per_token'] = df.usd_per_token_san
df.loc[df.usd_per_token_cmc.isnull() &
       ~df.usd_per_token_coinapi.isnull() &
       ~df.usd_per_token_coingecko.isnull() &
       ((df.usd_per_token_coinapi/df.usd_per_token_coingecko > 1.2) | 
        (df.usd_per_token_coinapi/df.usd_per_token_coingecko < 0.8)), 'usd_per_token'] = df.usd_per_token_coingecko
assert(0==df.usd_per_token.isnull().sum())

# Fix where usd_per_token_cmc is nonmissing
df['avg_usd_per_token'] = df[['usd_per_token_coinapi', 'usd_per_token_cmc',
                              'usd_per_token_coingecko', 'usd_per_token_san']].mean(axis=1)
df.loc[~df.usd_per_token_cmc.isnull() &
       (df.usd_per_token_cmc/df.avg_usd_per_token < 1.5) &
       (df.usd_per_token_cmc/df.avg_usd_per_token > 0.5), 'usd_per_token'] = df.avg_usd_per_token
assert(0==df.usd_per_token.isnull().sum())

# Drop created cols and old price columns
df = df.drop(['avg_usd_per_token', 'usd_per_token_coinapi', 'usd_per_token_cmc',
              'usd_per_token_coingecko', 'usd_per_token_san'], axis=1)

In [35]:
# CLEAN VOLUME COLUMNS

# Set any zero volumes to missing
df.loc[df.usd_volume_24h_coinapi == 0, 'usd_volume_24h_coinapi'] = np.nan
df.loc[df.usd_volume_24h_cmc == 0, 'usd_volume_24h_cmc'] = np.nan
df.loc[df.usd_volume_24h_coingecko == 0, 'usd_volume_24h_coingecko'] = np.nan
df.loc[df.usd_volume_24h_san == 0, 'usd_volume_24h_san'] = np.nan

# Create single volume olumn
df['usd_trading_volume_24h'] =  df[['usd_volume_24h_cmc', 'usd_volume_24h_coingecko',
                                    'usd_volume_24h_san']].mean(axis=1)

# Fix missing values
df.loc[df.usd_trading_volume_24h.isnull(), 'usd_trading_volume_24h'] = df.usd_volume_24h_coinapi

# Drop rows with missing
df = df[~df.usd_trading_volume_24h.isnull()].reset_index(drop=True)
assert(0==df.usd_trading_volume_24h.isnull().sum())

# Drop old volume columns
df = df.drop(['usd_volume_24h_coinapi', 'usd_volume_24h_cmc',
              'usd_volume_24h_coingecko', 'usd_volume_24h_san'], axis=1)

In [36]:
# CLEAN MCAP COLUMNS

# Set any zero mcap to missing
df.loc[df.usd_mcap_cmc == 0, 'usd_mcap_cmc'] = np.nan
df.loc[df.usd_mcap_coingecko == 0, 'usd_mcap_coingecko'] = np.nan
df.loc[df.usd_mcap_san == 0, 'usd_mcap_san'] = np.nan

# Create single mcap column
df['usd_mcap'] =  df[['usd_mcap_cmc', 'usd_mcap_coingecko', 'usd_mcap_san']].mean(axis=1)

# Drop rows with missing
df = df[~df.usd_mcap.isnull()].reset_index(drop=True)
assert(0==df.usd_mcap.isnull().sum())

# Drop old mcap columns
df = df.drop(['usd_mcap_cmc', 'usd_mcap_coingecko', 'usd_mcap_san'], axis=1)

In [37]:
# SORT COLUMNS
cols = list(df.columns.values)
first_cols = ['date', 'slug_cmc', 
              'usd_per_token', 'usd_mcap',
              'usd_trading_volume_24h']
for col in first_cols:
    cols.remove(col)
df = df[first_cols + cols]

### 4.3. Combine columns <a class="anchor" id="combine"></a>

In [38]:
# CUT DOWN TO UNIQUE COLUMNS

# Clean consensus_pow_messari and pow_cmc cols
df['consensus_pow'] = 0
df.loc[(df.consensus_pow_messari == 1) | (df.pow_cmc == 1), 'consensus_pow'] = 1
df = df.drop(['consensus_pow_messari', 'pow_cmc'], axis=1)

# Clean ETH fees columns
df = df.drop(['ethereum_average_fees_usd_san',
              'ethereum_fees_usd_san',
              'ethereum_median_fees_usd_san'], axis=1)

# Combine sector columns
sector_cols = list(df.filter(regex='sector', axis=1).columns.values)
df['sector_dex'] = 0
df.loc[(df.sector_san == 'Decentralized Exchange') |
       (df.sector_dex_messari == 1) |
       (df.sector_dex_cmc == 1), 'sector_dex'] = 1
df.loc[df.slug_cmc == 'serum', 'sector_dex'] = 1   
df['sector_cex'] = 0
df.loc[(df.sector_san.isin(['Exchange', 'Binance'])) |
       (df.sector_cex_cmc == 1), 'sector_cex'] = 1
df['sector_smart_contracts'] = 0
df.loc[(df.sector_san == 'Smart Contracts') |
       (df.sector_smart_contract_messari == 1), 'sector_smart_contracts'] = 1
df['sector_currency'] = 0
df.loc[(df.sector_san == 'Cryptocurrency') |
       (df.sector_currency_messari == 1) |
       (df.sector_moe_cmc == 1), 'sector_currency'] = 1
df.loc[df.slug_cmc == 'bitcoin', 'sector_currency'] = 1  
df.loc[df.slug_cmc == 'zcash', 'sector_currency'] = 1 
df.loc[df.slug_cmc == 'monero', 'sector_currency'] = 1 
df.loc[df.slug_cmc == 'litecoin', 'sector_currency'] = 1 
df.loc[df.slug_cmc == 'dash', 'sector_currency'] = 1 
df['sector_defi'] = 0
df.loc[(df.sector_san.isin(['Financial', 'DeFi', 'Lending', 'Trading'])) |
       (df.sector_defi_messari == 1) |
       (df.sector_defi_cmc == 1) |
       (df.sector_payment_messari == 1) | 
       (df.sector_payments_cmc == 1), 'sector_defi'] = 1
df.loc[df.slug_cmc == 'digitalbits', 'sector_defi'] = 1
df['sector_data'] = 0
df.loc[(df.sector_san.isin(['Data', 'Cloud Storage'])) |
       (df.sector_data_messari == 1), 'sector_data'] = 1
df['sector_games'] = 0
df.loc[(df.sector_san == 'Gaming') |
       (df.sector_games_messari == 1) |
       (df.sector_games_cmc == 1), 'sector_games'] = 1
df['sector_stablecoin'] = 0
df.loc[(df.sector_stablecoin_messari == 1) |
       (df.sector_stablecoin_cmc == 1), 'sector_stablecoin'] = 1
df['sector_compute'] = 0
df.loc[(df.sector_san == 'Cloud Computing') |
       (df.sector_compute_messari == 1) |
       (df.sector_computing_cmc == 1), 'sector_compute'] = 1
df['sector_metaverse'] = 0
df.loc[(df.sector_san == 'Virtual Reality') |
       (df.sector_san == 'Social') |
       (df.sector_social_messari == 1) |
       (df.sector_metaverse_messari == 1) |
       (df.sector_social_cmc == 1) |
       (df.sector_metaverse_cmc == 1), 'sector_metaverse'] = 1
df['sector_privacy'] = 0
df.loc[(df.sector_san == 'Privacy') |
       (df.sector_privacy_messari == 1) |
       (df.sector_privacy_cmc == 1), 'sector_privacy'] = 1
df.loc[df.slug_cmc == 'monero', 'sector_privacy'] = 1 
df.loc[df.slug_cmc == 'zcash', 'sector_privacy'] = 1 
df['sector_infra'] = 0
df.loc[(df.sector_san.isin(['Infrastructure', 'Interoperability', 'Verification', 'Masternodes'])) |
       (df.sector_infrastructure_messari == 1) |
       (df.sector_infastructure_cmc == 1), 'sector_infra'] = 1
df['sector_nft'] = 0
df.loc[(df.sector_nft_messari  == 1) |
       (df.sector_nft_cmc == 1), 'sector_nft'] = 1
df = df.drop(sector_cols, axis=1)

# Combine ecosystem variables
ecosystem_cols = list(df.filter(regex='eco', axis=1).columns.values)
ecosystem_cols.remove('ethereum_percent_of_whale_stablecoin_total_supply_san')
ecosystem_cols.remove('sector_stablecoin')
eco_vars = ['avalanche', 'binance', 'cosmos',  'eth',
            'polkadot', 'polygon', 'solana']
for eco_var in eco_vars:
    # Create new col
    df['ecosystem_'+eco_var] = 0
    
    # Update var
    df.loc[(df['ecosystem_'+eco_var+'_cmc'] == 1) |
           (df['ecosystem_'+eco_var+'_messari'] == 1), 'ecosystem_'+eco_var] =1
    
    # Set up slug var
    slug = eco_var
    if eco_var == 'binance':
        slug = 'bnb'
    if eco_var == 'eth':
        slug = 'ethereum'
    if eco_var == 'polkadot':
        slug = 'polkadot-new'

    # Ensure slug is at least set
    df.loc[df.slug_cmc == slug, 'ecosystem_'+eco_var] = 1
df = df.drop(ecosystem_cols, axis=1)

### 4.4. Interpolate <a class="anchor" id="interpolate"></a>


In [39]:
# INTERPOLATE TO FILL SHORT GAPS IN COIN HISTORY

# Ensure DF is properly sorted
df = df.sort_values(by=['date', 'slug_cmc'], ignore_index=True)
unique_tokens = list(np.unique(df.slug_cmc.values))

# Create new master df
new_df = pd.DataFrame()

# Loop over all assets
num_assets = len(unique_tokens)
for i in range(num_assets):
    # Report progress
    print('Completed ' + str(np.round(i/num_assets * 100, 2)) + '% of the assets.')

    # Obtain asset's obs, dates, and ensure unique
    asset = unique_tokens[i] 
    print(asset)
    temp_df = df[df.slug_cmc == asset]
    asset_dates = temp_df.date.values
    assert(len(asset_dates) == len(np.unique(asset_dates))) 

    # Determine all the date gaps for this asset
    gap_dates = {}
    for j in range(1,len(asset_dates)):
        gap_length = (asset_dates[j] - asset_dates[j-1]).astype('timedelta64[D]').astype(int)
        if gap_length > 1:
            gap_dates[asset_dates[j]] = gap_length
        else:
            assert(gap_length == 1)

    # If date gap is six days or less, then add these dates to the DF
    new_dates = []
    for k in range(len(gap_dates)):
        day = list(gap_dates.keys())[k]
        gap_length = list(gap_dates.values())[k]
        if gap_length <= 7:
            for l in range(gap_length-1, 0, -1):
                new_date = day - pd.Timedelta(l*86400000000000, unit ='ns') 
                new_dates.append(new_date)
    new_dates_df = pd.DataFrame(data={'date': new_dates,
                                     'slug_cmc': np.repeat(asset, len(new_dates))})
    nrow_b4 = temp_df.shape[0]
    temp_df = temp_df.merge(new_dates_df,
                            on=['date', 'slug_cmc'],
                            how='outer',
                            validate='one_to_one')
    assert(temp_df.shape[0]==(nrow_b4+new_dates_df.shape[0]))
    assert(new_dates_df.shape[0] <= np.sum(list(gap_dates.values())))

    # Interpolate values for this asset of gaps <= 6 days =; only forward in time
    temp_df = temp_df.sort_values(by=['date', 'slug_cmc'], ignore_index=True)
    temp_df = temp_df.set_index('date')
    temp_df = temp_df.interpolate(method='linear', limit=6, limit_direction='forward')
    temp_df = temp_df.reset_index()

    # If the first set of consecutive days isn't 34 days long, then drop that set of days
    while temp_df.shape[0] > 0:
        asset_dates = temp_df.date.values
        assert(len(asset_dates) == len(np.unique(asset_dates))) 
        last_date_of_first_consec_group = 'no gap'
        for m in range(1,len(asset_dates)):
            gap_length = (asset_dates[m] - asset_dates[m-1]).astype('timedelta64[D]').astype(int)
            if gap_length > 1:
                last_date_of_first_consec_group = asset_dates[m-1]
                break
            else:
                assert(gap_length == 1)
        if last_date_of_first_consec_group == 'no gap':
            last_date_of_first_consec_group = asset_dates[-1]
        length_first_consec_group = (last_date_of_first_consec_group - 
                                     asset_dates[0]).astype('timedelta64[D]').astype(int)
        if length_first_consec_group < 34:
            print('DROPPED FIRST BLOCK!')
            temp_df = temp_df[temp_df.date > last_date_of_first_consec_group]
        else:
            break

    # Drop any set of consecutive dates of 6 days or less
    asset_dates = temp_df.date.values
    assert(len(asset_dates) == len(np.unique(asset_dates))) 
    start_date = asset_dates[0]
    for n in range(1,len(asset_dates)):
        gap_length = (asset_dates[n] - asset_dates[n-1]).astype('timedelta64[D]').astype(int)
        if gap_length > 1: # detected block of days
            block_length = (asset_dates[n-1] - start_date).astype('timedelta64[D]').astype(int)
            if block_length <= 6:
                temp_df = temp_df[(temp_df.date < start_date) | (temp_df.date > asset_dates[n-1])]
            start_date = asset_dates[n] # reset start date to start of next block

    # Ensure critical columns are nonmissing
    assert(0==temp_df.date.isnull().sum())
    assert(0==temp_df.slug_cmc.isnull().sum())
    assert(0==temp_df.usd_per_token.isnull().sum())
    assert(0==temp_df.usd_mcap.isnull().sum())
    assert(0==temp_df.usd_trading_volume_24h.isnull().sum())

    # Add this asset's cleaned data to new DF
    new_df = pd.concat((new_df, temp_df))
    print('\n')
    
# Clean up new_df
df = new_df
del new_df
df = df.sort_values(by=['date', 'slug_cmc'], ignore_index=True)

Completed 0.0% of the assets.
0x


Completed 0.24% of the assets.
1inch


Completed 0.48% of the assets.
aave


Completed 0.72% of the assets.
aave-old


Completed 0.95% of the assets.
aavegotchi


Completed 1.19% of the assets.
abbc-coin


Completed 1.43% of the assets.
adbank


Completed 1.67% of the assets.
adshares


Completed 1.91% of the assets.
adx-net


Completed 2.15% of the assets.
aelf


Completed 2.39% of the assets.
aergo


Completed 2.63% of the assets.
aigang


Completed 2.86% of the assets.
aion


Completed 3.1% of the assets.
aioz-network


Completed 3.34% of the assets.
akropolis


Completed 3.58% of the assets.
alchemix


Completed 3.82% of the assets.
alchemy-pay


Completed 4.06% of the assets.
aleph-im


Completed 4.3% of the assets.
algorand


Completed 4.53% of the assets.
alien-worlds


Completed 4.77% of the assets.
allianceblock


Completed 5.01% of the assets.
alpha-finance-lab


Completed 5.25% of the assets.
alphacat


Completed 5.49% of the assets.
amber




Completed 44.87% of the assets.
iota


Completed 45.11% of the assets.
iotex


Completed 45.35% of the assets.
iungo


Completed 45.58% of the assets.
jibrel-network


Completed 45.82% of the assets.
just


Completed 46.06% of the assets.
kadena


Completed 46.3% of the assets.
kambria


Completed 46.54% of the assets.
kardiachain


Completed 46.78% of the assets.
kava


Completed 47.02% of the assets.
keep-network


Completed 47.26% of the assets.
kin


Completed 47.49% of the assets.
klever


Completed 47.73% of the assets.
komodo


Completed 47.97% of the assets.
kryll


Completed 48.21% of the assets.
kucoin-token


Completed 48.45% of the assets.
kusama


Completed 48.69% of the assets.
kyber-network-crystal-v2


Completed 48.93% of the assets.
lala-world


Completed 49.16% of the assets.
latoken


Completed 49.4% of the assets.
lattice-token


Completed 49.64% of the assets.
lcx


Completed 49.88% of the assets.
lisk


Completed 50.12% of the assets.
litecoin


Completed 50.36%



Completed 89.26% of the assets.
tribe


Completed 89.5% of the assets.
tron


Completed 89.74% of the assets.
truefi-token


Completed 89.98% of the assets.
trueflip


Completed 90.21% of the assets.
trust-wallet-token


Completed 90.45% of the assets.
ubix-network


Completed 90.69% of the assets.
ulord


Completed 90.93% of the assets.
uma


Completed 91.17% of the assets.
umbrella-network


Completed 91.41% of the assets.
unifi-protocol-dao


Completed 91.65% of the assets.
unifty


Completed 91.89% of the assets.
unilayer


Completed 92.12% of the assets.
uniswap


Completed 92.36% of the assets.
unmarshal-token


Completed 92.6% of the assets.
uquid-coin


Completed 92.84% of the assets.
utrust


Completed 93.08% of the assets.
v-systems


Completed 93.32% of the assets.
vechain


Completed 93.56% of the assets.
velas


Completed 93.79% of the assets.
verasity


Completed 94.03% of the assets.
vethor-token


Completed 94.27% of the assets.
videocoin


Completed 94.51% of the ass

### 4.5. Clean remaining columns <a class="anchor" id="columns"></a>


In [40]:
# ENSURE COINMETRICS COLUMNS ARE NON MISSING
cm_cols  = list(df.filter(regex='_cm', axis=1).columns.values)
cmc_cols = list(df.filter(regex='_cmc', axis=1).columns.values)
for cmc_col in cmc_cols:
    if cmc_col in cm_cols:
        cm_cols.remove(cmc_col)
for col in cm_cols:
    assert(0==df[col].isnull().sum()), print(col)

In [41]:
# CLEAN FED COLUMNS
fed_cols = list(df.filter(regex='_fed', axis=1).columns.values)
for col in fed_cols:
    assert(0==df[col].isnull().sum()), print(col)

In [42]:
# CLEAN THE COINGECKO COLS
coingecko_cols  =  list(df.filter(regex='_coingecko', axis=1).columns.values)

for col in coingecko_cols:
    # Set first value to zero if asset has nonmissing values eventually
    if col != 'alexa_rank_coingecko':
        slugs_with_non_missing_values = np.unique(df[~df[col].isnull()].slug_cmc.values)
        df.loc[df.groupby('slug_cmc')[col].head(1).index, 'first_ob'] = 1
        df.loc[(df.first_ob == 1) & (df[col].isnull()), 'first_ob_missing'] = 1
        df.loc[df.slug_cmc.isin(slugs_with_non_missing_values) & (df.first_ob_missing == 1), col] = 0
    else:
        # Fill initial values with first value
        df[col] = df.groupby('slug_cmc')[col].apply(lambda group: group.interpolate(method='linear',
                                                                                    limit_direction='backward',
                                                                                    limit_area='outside'))
    # Fill missings with linear interpolation
    df[col] = df.groupby('slug_cmc')[col].apply(lambda group: group.interpolate(method='linear',
                                                                                limit_direction='forward'))

df = df.drop(['first_ob', 'first_ob_missing'], axis=1)

In [43]:
# CLEAN MESSARI COLUMNS
messari_cols =  list(df.filter(regex='_messari', axis=1).columns.values)

# Drop columns
df = df.drop(['prct_premine_messari',
              'prct_founder_premine_messari'], axis=1)

# Clean columns of missing to set to zero
cols = ['supply_capped_messari','launch_crowdsale_messari',
        'launch_privatesale_messari', 'gov_no_on_chain_messari',
        'gov_on_chain_messari', 'token_usage_payments_messari',
        'token_usage_vote_messari', 'token_usage_work_messari', 
        'token_usage_access_messari', 'token_usage_discount_messari',
        'token_usage_dividends_messari', 'token_type_native_messari',
        'token_type_erc20_messari']
for col in cols:
    df.loc[df[col].isnull(), col] = 0
    assert(0==df[col].isnull().sum())

In [44]:
# CLEAN CMC COLUMNS

cmc_cols  =  list(df.filter(regex='_cmc', axis=1).columns.values)

# Clean zeros to missing
df.loc[df.total_supply_cmc == 0, 'total_supply_cmc'] = np.nan
df.loc[df.circulating_supply_cmc == 0, 'circulating_supply_cmc'] = np.nan

# Clean missing to zero
cols = ['mineable_cmc', 'portfolio_polychain_cmc',
        'portfolio_dcg_cmc', 'portfolio_usv_cmc', 
        'portfolio_3arrows_cmc', 'portfolio_placeholder_cmc',
        'portfolio_coinbase_cmc', 'portfolio_a16z_cmc',
        'portfolio_electric_cap_cmc', 'pos_cmc',
        'portfolio_multicoin_cap_cmc', 'portfolio_pantera_cmc', 
        'portfolio_dragonfly_cmc', 'portfolio_galaxy_digital_cmc',
        'platform_cmc', 'portfolio_alameda_cmc', 'portfolio_paradigm_cmc',
        'portfolio_winklevoss_cmc', 'exchange_volume_binance_us_cmc',
        'exchange_volume_coinbase_cmc', 'exchange_volume_gemini_cmc',
        'exchange_volume_kraken_cmc', 'exchange_volume_kucoin_cmc', 
        'exchange_pairs_binance_us_cmc', 'exchange_pairs_coinbase_cmc',
        'exchange_pairs_gemini_cmc', 'exchange_pairs_kraken_cmc',
        'exchange_pairs_kucoin_cmc']
for col in cols:
    df.loc[df[col].isnull(), col] = 0
    assert(0==df[col].isnull().sum())

# Interpolate
cols = ['num_market_pairs_cmc', 'circulating_supply_cmc',
        'total_supply_cmc', 'rank_cmc', 'first_date_cmc']
for col in cols:
    # Set first value to zero if asset has nonmissing values eventually
    if col in ['num_market_pairs_cmc']:
        slugs_with_non_missing_values = np.unique(df[~df[col].isnull()].slug_cmc.values)
        df.loc[df.groupby('slug_cmc')[col].head(1).index, 'first_ob'] = 1
        df.loc[(df.first_ob == 1) & (df[col].isnull()), 'first_ob_missing'] = 1
        df.loc[df.slug_cmc.isin(slugs_with_non_missing_values) & (df.first_ob_missing == 1), col] = 0
    else:
        # Fill initial values with first value
        df[col] = df.groupby('slug_cmc')[col].apply(lambda group: group.interpolate(method='linear',
                                                                                    limit_direction='backward',
                                                                                    limit_area='outside'))
    # Fill missings with linear interpolation
    df[col] = df.groupby('slug_cmc')[col].apply(lambda group: group.interpolate(method='linear',
                                                                                limit_direction='forward'))
    
    # Handle first_date
    if col == 'first_date_cmc':
        df[col] = df.groupby('slug_cmc')[col].apply(lambda group: group.interpolate(method='pad', limit=6))

# Clean up temp columns
df = df.drop(['first_ob', 'first_ob_missing'], axis=1)
   
# Clean up the missing values in first_date and drop last_date
df = df.drop('last_date_cmc', axis=1)
slugs_missing_first_date = np.unique(df[df.first_date_cmc.isnull()].slug_cmc.values)
for slug in slugs_missing_first_date:
    first_date = np.unique(df[(df.slug_cmc == slug) & ~df.first_date_cmc.isnull()].first_date_cmc.values)
    assert(1==len(first_date))
    first_date = first_date[0]
    df.loc[(df.slug_cmc == slug) & df.first_date_cmc.isnull(), 'first_date_cmc'] = first_date

# Drop a suspect token
df = df[df.slug_cmc != 'kyber-network-crystal-v2']

# Confirm first_date_cmc is nonmissing and all before data
assert(0==df[df.first_date_cmc.isnull()].shape[0])
assert(0==df[(pd.to_datetime(df.date) - pd.to_datetime(df.first_date_cmc)).astype('timedelta64[D]') < 0].shape[0])

# Clean total_volume_24h_reported_cmc by setting first value and then cubic interpolate
df.loc[df.date == '2015-01-01', 'total_volume_24h_reported_cmc'] = 36e6 #note: from way back machine
df['total_volume_24h_reported_cmc'] = df.total_volume_24h_reported_cmc.interpolate(method='cubic')

# Clean altcoin_volume_24h_reported_cmc by setting first value and then cubic interpolate
df.loc[df.date == '2015-01-01', 'altcoin_volume_24h_reported_cmc'] = 6e6 #note: from way back machine
df['altcoin_volume_24h_reported_cmc'] = df.altcoin_volume_24h_reported_cmc.interpolate(method='cubic')

# Clean active_cryptocurrencies_cmc by setting first value and then cubic interpolate
df.loc[df.date == '2015-01-01', 'active_cryptocurrencies_cmc'] = 500 #note: from way back machine
df['active_cryptocurrencies_cmc'] = df.active_cryptocurrencies_cmc.interpolate(method='cubic')

# Clean active_market_pairs_cmc by setting first value and then cubic interpolate
df.loc[df.date == '2015-01-01', 'active_market_pairs_cmc'] = 1500 #note: from way back machine
df['active_market_pairs_cmc'] = df.active_market_pairs_cmc.interpolate(method='cubic')

# Clean active_exchanges_cmc by setting first value and then cubic interpolate
df.loc[df.date == '2015-01-01', 'active_exchanges_cmc'] = 60 #note: from way back machine
df['active_exchanges_cmc'] = df.active_exchanges_cmc.interpolate(method='cubic')

In [45]:
# CLEAN SAN COLUMNS

san_cols = list(df.filter(regex='_san', axis=1).columns.values)

# Clean missing to zero
cols = ['ethereum_defi_total_value_locked_usd_san',
        'ethereum_nft_trade_volume_usd_san',
        'ethereum_nft_trades_count_san',
        'ethereum_nft_retail_trade_volume_usd_san',
        'ethereum_nft_whale_trade_volume_usd_san',
        'ethereum_nft_whale_trades_count_san',
        'ethereum_percent_of_whale_stablecoin_total_supply_san',
        'bitcoin_miners_to_exchanges_flow_san',
        'ethereum_miners_to_exchanges_flow_san',
        'ethereum_miners_exchange_balance_san',
        'ethereum_traders_to_defi_flow_san',
        'ethereum_traders_defi_balance_san',
        'ethereum_stock_to_flow_san']
for col in cols:
    df.loc[df[col].isnull(), col] = 0
    assert(0==df[col].isnull().sum())
    
# Interpolate
cols = ['active_addresses_24h_san', 'age_consumed_san', 'age_destroyed_change_7d_san',
        'circulation_san', 'daily_active_addresses_san', 'dev_activity_san', 
        'dev_activity_1d_san', 'dev_activity_contributors_count_san', 
        'dormant_circulation_90d_san', 'github_activity_san',
        'github_activity_contributors_count_san', 
        'mean_age_san', 'mean_dollar_invested_age_san', 'mean_realized_price_usd_san',
        'mvrv_usd_san', 'nvt_san', 'nvt_transaction_volume_san', 'payments_count_san',
        'rank_san', 'realized_value_usd_san', 'sentiment_balance_bitcointalk_san',
        'sentiment_balance_reddit_san', 'sentiment_balance_telegram_san',
        'sentiment_balance_total_san', 'sentiment_negative_bitcointalk_san',
        'sentiment_negative_reddit_san', 'sentiment_negative_telegram_san',
        'sentiment_negative_total_san', 'sentiment_positive_bitcointalk_san', 
        'sentiment_positive_reddit_san', 'sentiment_positive_telegram_san',
        'sentiment_positive_total_san', 'sentiment_volume_consumed_bitcointalk_san',
        'sentiment_volume_consumed_reddit_san', 'sentiment_volume_consumed_telegram_san',
        'sentiment_volume_consumed_total_san', 'social_dominance_bitcointalk_san',
        'social_dominance_reddit_san', 'social_dominance_telegram_san', 'social_dominance_total_san',
        'social_volume_bitcointalk_san', 'social_volume_reddit_san', 'social_volume_telegram_san',
        'social_volume_total_san', 'supply_on_exchanges_san', 'supply_outside_exchanges_san',
        'total_supply_in_profit_san', 'transaction_volume_san', 'transaction_volume_usd_san',
        'transactions_count_san', 'velocity_san']
for col in cols:
    # Set first value to zero if asset has nonmissing values eventually
    if col not in ['mean_age_san', 'mean_dollar_invested_age_san',
                   'mean_realized_price_usd_san', 'mvrv_usd_san',
                   'rank_san', 'realized_value_usd_san', 'supply_on_exchanges_san',
                   'supply_outside_exchanges_san', 'total_supply_in_profit_san']:
        slugs_with_non_missing_values = np.unique(df[~df[col].isnull()].slug_cmc.values)
        df.loc[df.groupby('slug_cmc')[col].head(1).index, 'first_ob'] = 1
        df.loc[(df.first_ob == 1) & (df[col].isnull()), 'first_ob_missing'] = 1
        df.loc[df.slug_cmc.isin(slugs_with_non_missing_values) & (df.first_ob_missing == 1), col] = 0
    else:
        # Fill initial values with first value
        df[col] = df.groupby('slug_cmc')[col].apply(lambda group: group.interpolate(method='linear',
                                                                                    limit_direction='backward',
                                                                                    limit_area='outside'))
    # Fill missings with linear interpolation
    df[col] = df.groupby('slug_cmc')[col].apply(lambda group: group.interpolate(method='linear',
                                                                                limit_direction='forward'))

# Clean up
df = df.drop(['first_ob', 'first_ob_missing'], axis=1)

# Clean macro col
df.loc[df.date == '2015-01-01', 'token_mcap_avg_mcd_collat_ratio_san'] = np.min(df.token_mcap_avg_mcd_collat_ratio_san)
df['token_mcap_avg_mcd_collat_ratio_san'] = df['token_mcap_avg_mcd_collat_ratio_san'].interpolate(method='cubic')

# Clean macro col
df.loc[df.date == '2015-01-01', 'ethereum_mvrv_usd_intraday_san'] = np.min(df.ethereum_mvrv_usd_intraday_san)
df['ethereum_mvrv_usd_intraday_san'] = df['ethereum_mvrv_usd_intraday_san'].interpolate(method='cubic')

# Take average of daily_active_addresses_san and active_addresses_24h_san to form active_addresses_san
df['active_addresses_san'] = df[['daily_active_addresses_san',
                                 'active_addresses_24h_san']].mean(axis=1)
df = df.drop(['daily_active_addresses_san', 'active_addresses_24h_san'], axis=1)

# Take average of dev_activity_1d_san and dev_activity_san to form dev_activ_san
df['dev_activ_san'] = df[['dev_activity_1d_san', 'dev_activity_san']].mean(axis=1)
df = df.drop(['dev_activity_1d_san', 'dev_activity_san'], axis=1)

In [46]:
# EDIT A PRICE VALUE
df.loc[(df.slug_cmc == 'skale-network') & (df.date == '2020-12-01'), 'usd_per_token'] = 0.76


In [47]:
# ENSURE NO DUPLICATED ROWS
assert(0==df[df.duplicated(subset=['date', 'slug_cmc'])].shape[0])

### 4.6. Subset to token universe <a class="anchor" id="subset"></a>

In [48]:
# LOOP OVER TOKEN UNIVERSE TO SUBSET DF TO VALID ROWS
included_df = pd.DataFrame()

# Loop over asset_universe_dict len with i
for i in range(len(asset_universe_dict)):
    # Extract this quarter and its included assets
    date = list(asset_universe_dict.keys())[i]
    assets = asset_universe_dict[date]

    # Form end date for this window
    end_date   = datetime.strptime(date, '%Y-%m-%d') + relativedelta(months=3)

    # Extract asset dates
    temp_df = df[(df.slug_cmc.isin(assets)) & (df.date < end_date)][['date', 'slug_cmc']]
    included_df = pd.concat((included_df, temp_df.reset_index(drop=True)))

# Add the 2022 date-assets to form last week of 2021 future return
date = '2021-10-01'
assets = asset_universe_dict[date]
temp_df = df[(df.slug_cmc.isin(assets)) & (df.date.dt.year == 2022)][['date', 'slug_cmc']]
included_df = pd.concat((included_df, temp_df.reset_index(drop=True)))

# Keep just unique rows
included_df = included_df.drop_duplicates()
assert(0==included_df[included_df.duplicated()].shape[0])
df = df.merge(included_df,
              on=['date', 'slug_cmc'],
              how='inner',
              validate='one_to_one')

### 4.7. Final cleaning steps and save <a class="anchor" id="save"></a>

In [49]:
# ENSURE NO DUPLICATED ROWS
assert(0==df[df.duplicated(subset=['date', 'slug_cmc'])].shape[0])

In [50]:
# SORT

# Sort columns
cm_cols  = list(df.filter(regex='_cm', axis=1).columns.values)
cmc_cols = list(df.filter(regex='_cmc', axis=1).columns.values)
for cmc_col in cmc_cols:
    if cmc_col in cm_cols:
        cm_cols.remove(cmc_col)
cmc_cols.remove('slug_cmc')
coingecko_cols = list(df.filter(regex='_coingecko', axis=1).columns.values)
messari_cols = list(df.filter(regex='_messari', axis=1).columns.values)
san_cols = list(df.filter(regex='_san', axis=1).columns.values)
fed_cols = list(df.filter(regex='_fed', axis=1).columns.values)
new_col_order = (['date', 'slug_cmc', 'usd_per_token', 
                  'usd_mcap', 'usd_trading_volume_24h'] 
                 + cm_cols + coingecko_cols + cmc_cols  
                 + san_cols + messari_cols + fed_cols +
                 ['consensus_pow', 'sector_dex', 'sector_cex',
                  'sector_smart_contracts', 'sector_currency', 'sector_defi',
                  'sector_data', 'sector_games', 'sector_stablecoin',
                  'sector_compute', 'sector_metaverse', 'sector_privacy',
                  'sector_infra', 'sector_nft', 'ecosystem_avalanche',
                  'ecosystem_binance', 'ecosystem_cosmos', 'ecosystem_eth',
                  'ecosystem_polkadot', 'ecosystem_polygon', 'ecosystem_solana'])
assert(len(new_col_order) == len(df.columns.values))
df = df[new_col_order]

# Sort rows
df = df.sort_values(by=['date', 'slug_cmc'], ignore_index=True)

# Rename columns
df = df.rename(columns={'slug_cmc': 'asset'})

In [51]:
# OUTPUT CLEAN PANEL AND TOKEN UNIVERSE
df.to_pickle('../3-data/clean/raw_panel.pkl')

In [52]:
# SEND TO JAKE FOR THE CORRELATIONS TABLE
output_df = output_df[['date',
                       'asset',
                       'usd_per_token',
                       'usd_mcap',
                       'sector_dex',
                       'sector_cex',
                       'sector_smart_contracts',
                       'sector_currency',
                       'sector_defi',
                       'sector_data',
                       'sector_games',
                       'sector_stablecoin',
                       'sector_compute',
                       'sector_metaverse',
                       'sector_privacy',
                       'sector_infra',
                       'sector_nft']]
output_df.to_pickle('../raw_panel.pkl')

NameError: name 'output_df' is not defined

In [ ]:
# SEND TO CHUHAN AND LET HIM KNOW OUR TIME PERIOD IS 2016 to 2021 not starting in 2015
output_df = df[['date', 'asset', 'usd_per_token', 'usd_mcap']]
output_df.to_pickle('../raw_panel.pkl')